<a href="https://colab.research.google.com/github/SalmanulFarisKA/Python-Projects/blob/main/Dog_vs_Cat_Classification_Transfer_learning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing kaggle library

!pip install kaggle

In [2]:
# configuring the path of Kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing Dog vs Cat dataset from kaggle

In [3]:
!kaggle competitions download -c dogs-vs-cats

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [4]:
!ls

sample_data


In [5]:
# extracting the compressed file

from zipfile import ZipFile

dataset = '/content/dogs-vs-cats.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()

print('The dataset is extracted')

FileNotFoundError: [Errno 2] No such file or directory: '/content/dogs-vs-cats.zip'

In [ ]:
# extracting the compressed file

from zipfile import ZipFile

dataset = '/content/train.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()

print('The dataset is extracted')

In [ ]:
import os

# counting the number of files in train folder

path, dirs, files = next(os.walk('/content/train'))
file_count = len(files)

print(file_count)

Printing the name of images

In [ ]:
file_names = os.listdir('/content/train')
print(file_names)

Importing the dependencies

In [ ]:
!pip install google.colab

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

Displaying images

In [ ]:
img = mpimg.imread('/content/train/dog.3951.jpg')
imgplt = plt.imshow(img)
plt.show()

In [ ]:
img = mpimg.imread('/content/train/cat.10998.jpg')
imgplt = plt.imshow(img)
plt.show()

In [ ]:
# finding the number of cat and dog images

file_names = os.listdir('/content/train/')

dog_count = 0
cat_count = 0

for img_file in file_names:

  name = img_file[0:3]

  if name == 'dog':
    dog_count += 1

  else:
    cat_count += 1

print('Number of dog images =', dog_count)
print('Number of cat images =', cat_count)

Resizing all the images

In [ ]:
# to get a copy of an image and resized image

# creating a directory of these resized images

os.mkdir('/content/image resized')

In [ ]:
original_folder = '/content/train/'
resized_folder = '/content/image resized/'


# to take just 2000 first pictures

for i in range(2000):

  filename = os.listdir(original_folder)[i]
  img_path = original_folder+filename # /content/train/+cat.10998.jpg

  # open the image and resize it
  img = Image.open(img_path)
  img = img.resize((224, 224))
  img = img.convert('RGB')

  newImgPath = resized_folder+filename
  img.save(newImgPath)

In [ ]:
# finding the number of cat and dog images

file_names = os.listdir('/content/image resized/')

dog_count = 0
cat_count = 0

for img_file in file_names:

  name = img_file[0:3]

  if name == 'dog':
    dog_count += 1

  else:
    cat_count += 1

print('Number of dog images =', dog_count)
print('Number of cat images =', cat_count)

In [ ]:
# display resized image

img = mpimg.imread('/content/image resized/cat.10998.jpg')
imgplt = plt.imshow(img)
plt.show()

Creating labels for resized images

cat -> 0
dog -> 1

In [ ]:
# creating a for loop to assign labels to the images

filenames = os.listdir('/content/image resized/')

labels = []

for i in range(2000):

  file_name = filenames[i]
  label = file_name[0:3]

  if label == 'dog':
    labels.append(1)
  else:
    labels.append(0)

In [ ]:
print(filenames[0:5])
print(labels[0:5])
print(len(filenames))

In [ ]:
# finding the count of dogs and cats in a different way

values, counts = np.unique(labels, return_counts=True)

In [ ]:
print(values)
print(counts)

Converting to numpy arrays

In [ ]:
import cv2
import glob

In [ ]:
image_directory = '/content/image resized/'
image_extension = ['png', 'jpg']

files = []

# locate the file considering both jpg and png format, append it to files]
[files.extend(glob.glob(image_directory + '*.' + e)) for e in image_extension]

# take every individual image (as a numpy array) and convert it to one numpy array
dog_cat_images = np.asarray([cv2.imread(file) for file in files])

In [ ]:
print(dog_cat_images)

In [ ]:
type(dog_cat_images)

In [ ]:
print(dog_cat_images.shape)

In [ ]:
X = dog_cat_images
Y = np.asarray(labels)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [ ]:
# scaling the data (rgb values range from 0 to 255)

X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
print(X_train_scaled[0:5])

Building the Neural Network

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
mobilenet_model = 'https://www.kaggle.com/models/google/mobilenet-v2/frameworks/tensorFlow2/variations/tf2-preview-feature-vector/versions/4?tfhub-redirect=true'
pretrained_model = hub.KerasLayer(mobilenet_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
num_of_classes = 2

model = tf.keras.Sequential(
    [
      pretrained_model,
      tf.keras.layers.Dense(num_of_classes)
    ]
)

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

In [ ]:
model.fit(X_train_scaled, Y_train, epochs=5)

In [ ]:
score, acc = model.evaluate(X_test_scaled, Y_test)
print('Test Loss =', score)
print('Test Accuracy =', acc)

Predictive System

In [ ]:
input_image_path = input('Path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resize = cv2.resize(input_image, (224, 224))

input_image_scaled = input_image_resize/255

image_reshaped = np.reshape(input_image_scaled, [1, 224, 224, 3])
# to tell the model that im making prediction for only one image (as opposed to [2000, 224, 224, 3])

input_prediction = model.predict(image_reshaped)

input_pred_label = np.argmax(input_prediction)

if input_pred_label == 0:
  print("The image represents a cat")
else:
  print("The image represents a dog")